In [1]:
from datascience import *
import numpy as np

# Lecture 11 #

## Joins

In [2]:
drinks = Table().with_columns(
    'Drink', make_array('Milk Tea', 'Espresso', 'Latte', 'Espresso'),
    'Cafe', make_array('Asha', 'Strada', 'Strada', 'FSM'),
    'Price', make_array(5.5, 1.75, 3.25, 2)
)

drinks

Drink,Cafe,Price
Milk Tea,Asha,5.5
Espresso,Strada,1.75
Latte,Strada,3.25
Espresso,FSM,2


In [3]:
discounts = Table().with_columns(
    'Coupon % off', make_array(10, 25, 5),
    'Location', make_array('Asha', 'Strada', 'Asha')
)
discounts

Coupon % off,Location
10,Asha
25,Strada
5,Asha


In [4]:
combined = drinks.join('Cafe', discounts, 'Location')
combined

Cafe,Drink,Price,Coupon % off
Asha,Milk Tea,5.5,10
Asha,Milk Tea,5.5,5
Strada,Espresso,1.75,25
Strada,Latte,3.25,25


In [5]:
discounted_frac = 1 - combined.column('Coupon % off') / 100

combined.with_column(
    'Discounted Price', 
    combined.column('Price') * discounted_frac
)

Cafe,Drink,Price,Coupon % off,Discounted Price
Asha,Milk Tea,5.5,10,4.95
Asha,Milk Tea,5.5,5,5.225
Strada,Espresso,1.75,25,1.3125
Strada,Latte,3.25,25,2.4375


In [6]:
drinks.join('Cafe', drinks, 'Cafe')

Cafe,Drink,Price,Drink_2,Price_2
Asha,Milk Tea,5.5,Milk Tea,5.5
FSM,Espresso,2,Espresso,2
Strada,Espresso,1.75,Espresso,1.75
Strada,Espresso,1.75,Latte,3.25
Strada,Latte,3.25,Espresso,1.75
Strada,Latte,3.25,Latte,3.25


## Bigger Joins

### What were the top 10 states with the lowest population per square mile in the year 2000?

In [7]:
state_areas = Table.read_table('state-areas.csv')
state_pop = Table.read_table('state-population.csv')
state_abbrevs = Table.read_table('state-abbrevs.csv')

state_areas.show(5), state_pop.show(5), state_abbrevs.show(5);

state,area (sq. mi)
Alabama,52423
Alaska,656425
Arizona,114006
Arkansas,53182
California,163707


state/region,ages,year,population
AL,under18,2012,1.11749e+06
AL,total,2012,4.81753e+06
AL,under18,2010,1.13097e+06
AL,total,2010,4.78557e+06
AL,under18,2011,1.12576e+06


state,abbreviation
Alabama,AL
Alaska,AK
Arizona,AZ
Arkansas,AR
California,CA


In [8]:
state_pop_total_2000 = state_pop.where('ages', 'total').where('year', 2000)
state_pop_total_2000

state/region,ages,year,population
AL,total,2000,4.45217e+06
AK,total,2000,627963
AZ,total,2000,5.16059e+06
AR,total,2000,2.67859e+06
CA,total,2000,3.3988e+07
CO,total,2000,4.32692e+06
CT,total,2000,3.41178e+06
DE,total,2000,786373
DC,total,2000,572046
FL,total,2000,1.60475e+07


In [9]:
state_combined = state_pop_total_2000.join('state/region', state_abbrevs, 'abbreviation')
state_combined

state/region,ages,year,population,state
AK,total,2000,627963,Alaska
AL,total,2000,4.45217e+06,Alabama
AR,total,2000,2.67859e+06,Arkansas
AZ,total,2000,5.16059e+06,Arizona
CA,total,2000,3.3988e+07,California
CO,total,2000,4.32692e+06,Colorado
CT,total,2000,3.41178e+06,Connecticut
DC,total,2000,572046,District of Columbia
DE,total,2000,786373,Delaware
FL,total,2000,1.60475e+07,Florida


In [10]:
state_combined2 = state_combined.join('state', state_areas, 'state')
state_combined2

state,state/region,ages,year,population,area (sq. mi)
Alabama,AL,total,2000,4.45217e+06,52423
Alaska,AK,total,2000,627963,656425
Arizona,AZ,total,2000,5.16059e+06,114006
Arkansas,AR,total,2000,2.67859e+06,53182
California,CA,total,2000,3.3988e+07,163707
Colorado,CO,total,2000,4.32692e+06,104100
Connecticut,CT,total,2000,3.41178e+06,5544
Delaware,DE,total,2000,786373,1954
District of Columbia,DC,total,2000,572046,68
Florida,FL,total,2000,1.60475e+07,65758


In [11]:
state_combined3 = state_combined2.with_column(
    'Pop/sq mile',
    state_combined2.column('population') / state_combined2.column('area (sq. mi)')
)

state_combined3

state,state/region,ages,year,population,area (sq. mi),Pop/sq mile
Alabama,AL,total,2000,4.45217e+06,52423,84.9279
Alaska,AK,total,2000,627963,656425,0.956641
Arizona,AZ,total,2000,5.16059e+06,114006,45.2659
Arkansas,AR,total,2000,2.67859e+06,53182,50.3664
California,CA,total,2000,3.3988e+07,163707,207.615
Colorado,CO,total,2000,4.32692e+06,104100,41.565
Connecticut,CT,total,2000,3.41178e+06,5544,615.4
Delaware,DE,total,2000,786373,1954,402.443
District of Columbia,DC,total,2000,572046,68,8412.44
Florida,FL,total,2000,1.60475e+07,65758,244.039


In [12]:
state_combined3.sort('Pop/sq mile').take(np.arange(10))

state,state/region,ages,year,population,area (sq. mi),Pop/sq mile
Alaska,AK,total,2000,627963,656425,0.956641
Wyoming,WY,total,2000,494300,97818,5.05326
Montana,MT,total,2000,903773,147046,6.14619
North Dakota,ND,total,2000,642023,70704,9.08043
South Dakota,SD,total,2000,755844,77121,9.80075
New Mexico,NM,total,2000,1.8212e+06,121593,14.9779
Idaho,ID,total,2000,1.29943e+06,83574,15.5483
Nevada,NV,total,2000,2.01874e+06,110567,18.2581
Nebraska,NE,total,2000,1.71382e+06,77358,22.1544
Utah,UT,total,2000,2.2445e+06,84904,26.4358
